In [ ]:
import cv2
import numpy as np
import os
from glob import glob
from os.path import join
import tarfile
import urllib   # 웹에서 데이터를 다운로드 받을 때 사용

from matplotlib import pyplot as plt
import tensorflow as tf
from google.colab.patches import cv2_imshow

import copy
import time

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 텍스트 영역 인식


In [ ]:
# 이미지 리사이즈

def resize_image(img, new_width=1000):
    if img is None:
        print(f"Failed to load image")
        return

    height, width, channels = img.shape
    new_height = int((new_width / width) * height)

    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_img

In [ ]:
# 얼굴 영역 인식

cascade_filename = "" #가중치 파일 경로
cascade = cv2.CascadeClassifier(cascade_filename) #모델 불러오기

SCALE_FACTOR = 1.05
MIN_NEIGHBORS = 3
FACE_DENOMINATOR = 8

def imgDetector(img,cascade, file_name):
    # 그레이 스케일 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    min_face_size = img.shape[1]//FACE_DENOMINATOR
    results = cascade.detectMultiScale(gray, scaleFactor= SCALE_FACTOR, minNeighbors=MIN_NEIGHBORS, minSize=(min_face_size, min_face_size))
    if type(results) == tuple:
        print("ERROR: no face detect for " + file_name)
    return results[0]

def boxFace(img, file_name) :
    points = imgDetector(img, cascade, file_name)
    x, y, w, h = points
    w_offset = w//6
    h_offset = h//6
    newPoints = [x+w_offset, y-h_offset, w-(2*w_offset), h+(2*h_offset)]
    return newPoints

def drawBox(img, boxes, r, g, b) :
    clone = img.copy()
    for box in boxes :
        x, y, w, h = box
        cv2.rectangle(clone, (x,y), (x+w, y+h), (b,g,r), thickness=5)
    cv2_imshow(clone)

In [ ]:
# 텍스트 영역 인식

def kernelSize(img, denominator) :
    w = img.shape[0]
    kSize = w//denominator
    if (kSize % 2 == 0):
        kSize += 1
    return kSize


def extractText(img, denominator) :
    start = time.time()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kSize = kernelSize(img, denominator)

    gray = cv2.GaussianBlur(gray, (kSize, kSize), 0)

    mser = cv2.MSER_create()
    regions,_ = mser.detectRegions(gray)

    hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]

    tmp = list()
    count = 0
    for j, cnt in enumerate(hulls):
        x, y, w, h = cv2.boundingRect(cnt)
        tmp.append([x, y, w, h])
        count+=1
    return tmp

def drawTextBox(img, boxes) :
    clone = img.copy()
    for box in boxes :
        cv2.rectangle(clone, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (0, 255,0), 2)
    cv2_imshow(clone)

In [ ]:
# 박스 그리기 + 영역 계산

def detectIntersection(textBoxes, faceBox) :
    Fx, Fy, Fw, Fh = faceBox
    result = list()
    for box in textBoxes :
        x, y, w, h = box
        p1 = [x, y]
        p2 = [x+w, y]
        p3 = [x, y+h]
        p4 = [x+w, y+h]
        tmp = [p1, p2, p3, p4]
        for point in tmp :
            if (point[0] >= Fx and point[0] <= Fx+Fw and point[1] >= Fy and point[1] <= Fy+Fh) :
                break
        else :
            result.append(box)
    return result

def drawBox(img, boxes) :
    clone = img.copy()
    for box in boxes :
        x, y, w, h = box
        cv2.rectangle(clone, (x,y), (x+w, y+h), (0,255,0), thickness=1)
    cv2_imshow(clone)

def green_rectangles(img, boxes):
    clone = img.copy()
    for rect in boxes:
        x, y, w, h = rect
        clone[y:y+h, x:x+w] = (0, 255, 0)

    return clone

## 얼굴 인식 함수

In [ ]:
class DeepLabModel(object):
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        self.graph = tf.Graph()
        graph_def = None
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        with self.graph.as_default():
    	    tf.compat.v1.import_graph_def(graph_def, name='')

        self.sess = tf.compat.v1.Session(graph=self.graph)

    def preprocess(self, img_orig):
        height, width = img_orig.shape[:2]
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = cv2.resize(img_orig, target_size)
        resized_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        img_input = resized_rgb
        return img_input

    def run(self, image):
        img_input = self.preprocess(image)

        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [img_input]})

        seg_map = batch_seg_map[0]
        return cv2.cvtColor(img_input, cv2.COLOR_RGB2BGR), seg_map

In [ ]:
MODEL = DeepLabModel("") # Deeplab 모델 경로

LABEL_NAMES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]
len(LABEL_NAMES)

In [ ]:
# 추출된 영역 다듬기

def filter_small_mask_areas(mask, min_area_threshold):
    # 마스크 영역의 바이너리 이미지에서 연결된 컴포넌트를 찾음
    # 바이너리 마스크에서 연결된 컴포넌트 찾기
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 마스크에서 작은 영역을 제거하기 위한 마스크 생성
    filtered_mask = np.zeros_like(mask)

    # 마스크에서 컨투어 영역의 넓이를 계산하고, 일정 크기 이상인 경우만 유지

    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_area_threshold:
            contour_mask = np.zeros_like(mask)
            cv2.drawContours(contour_mask, [contour], 0, 255, -1)

            contour_pixels = mask[contour_mask == 255]
            average_color = contour_pixels[0]
            if average_color != 0:
                cv2.drawContours(filtered_mask, [contour], 0, 255, -1)

    return filtered_mask



def adjust_mask(img_mask_up, boxed):
    # 원본 이미지에서 초록색 픽셀 위치 찾기
    black_indices = np.where(np.all(boxed == [0, 255, 0], axis=-1))

    for i in range(len(black_indices[0])):
        y, x = black_indices[0][i], black_indices[1][i]
        img_mask_up[y, x] = 0

    return img_mask_up

def adjust_mask2(img_mask_up, bar) :
    adjusted_mask = img_mask_up.copy()

    height, width = adjusted_mask.shape
    for y in range(bar, height):
        for x in range(width):
            if adjusted_mask[y, x] == 255:
                adjusted_mask[y, x] = 0

    return adjusted_mask


def green_mask(boxed, original) :
    img_resized, seg_map = MODEL.run(original)
    seg_map = np.where(seg_map == 15, 15, 0) # 예측 중 사람만 추출
    img_mask = seg_map * (255/seg_map.max()) # 255 normalization
    img_mask = img_mask.astype(np.uint8)

    img_mask_up = cv2.resize(img_mask, original.shape[:2][::-1], interpolation=cv2.INTER_LINEAR)
    _, img_mask_up = cv2.threshold(img_mask_up, 128, 255, cv2.THRESH_BINARY)


    img_mask_up = adjust_mask(img_mask_up, boxed)
    img_mask_up = adjust_mask2(img_mask_up, 1000)
    img_mask_up = filter_small_mask_areas(img_mask_up, 10000)


    mask_color = (0, 255, 0)  # 변경할 색상 설정
    masked_img = np.zeros_like(original)  # 원본 이미지와 동일한 크기의 검은색 이미지 생성
    masked_img[img_mask_up == 255] = mask_color
    masked_img[img_mask_up == 0] = original[img_mask_up == 0]

    return masked_img

## 실행 코드

In [ ]:
input_folder = ""
output_folder = ""

dir_list = [] # 마스킹할 이미지가 담긴 폴더 리스트
count = 0

for path in dir_list :
    new_path = output_folder + path
    os.makedirs(new_path, exist_ok=True)

    print(f"Directory created: {new_path}")

for name in dir_list :
    print()
    print()
    print(f"new folder: {name}")
    input_dir = input_folder + name
    output_dir = output_folder + name

    file_list = os.listdir(input_dir)

    images = []
    image_num = 0

    for file_name in file_list:
        img_path = os.path.join(input_dir, file_name)
        image = cv2.imread(img_path)
        if image is not None:
            images.append([file_name, resize_image(image)])
            count += 1
            if (count % 20 == 0) :
                print(count)

    print("불러온 이미지 개수:", len(images))

    resized_images = images

    for i in range(len(resized_images)):

        if (i%10 == 0) :
            print(f"currenty at {i}")

        try :

            filename = resized_images[i][0]
            image = resized_images[i][1]

            faceBox = boxFace(image, filename)
            textBoxes = extractText(image, 150)
            newTextBoxes = detectIntersection(textBoxes, faceBox)
            boxed = green_rectangles(image, newTextBoxes)

            result = green_mask(boxed, image)

            file_path = os.path.join(output_dir, filename)
            cv2.imwrite(file_path, result)

        except Exception as e:
            print(f"Error occurred at index {i}: {e}")